In [1]:
import torch

In [2]:
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: mps


In [3]:
import numpy as np
import pandas as pd
import torch.nn as nn
import os
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import utils

In [4]:
# load all files into one single df
df = pd.concat([pd.read_csv('hka-aqm-am/' + f, skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])

In [5]:
train_df, test_df, train_loader, test_loader, scaler, y_test = utils.get_data_for_transformer(df, aggregation_level='quarter_hour', window_size=20)

/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['CO2_scaled'] = scaler.fit_transform(df_train[['CO2']])
/Users/timehmann/Library/Mobile Documents/com~apple~CloudDocs/Studium/Data_Science_Karlsruhe/Data_Science_Semester6/Internet_Of_Things/IoT_Project/utils.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['CO2_scaled'] = scaler.tr

Training data shape: torch.Size([245095, 20, 1]) torch.Size([245095, 1])
Testing data shape: torch.Size([63746, 20, 1]) torch.Size([63746, 1])


In [6]:
# print number of training samples
print(f"Number of training samples: {len(train_df)}")

Number of training samples: 245095


In [7]:
trained_model = utils.train_transformer_model(device, train_loader, test_loader, epochs=10)

TypeError: train_transformer_model() got multiple values for argument 'device'

In [ ]:
utils.evaluate_transformer_model(device, test_loader, trained_model, scaler, y_test)

In [ ]:
df.describe()

,tmp,hum,CO2,VOC,vis,IR,WIFI,BLE,rssi,channel_rssi,channel_index,spreading_factor,bandwidth,f_cnt
count,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.0,608036.000000
mean,25.142613,36.724730,607.723873,925.523094,801.135497,227.006169,2.476212,5.361508,-95.298145,-95.298145,3.497678,7.568132,125000.0,7406.347787
std,4.123532,9.568186,1058.630803,1161.608153,3225.759145,1355.515628,1.743450,11.378924,21.379657,21.379657,2.292534,0.683923,0.0,7658.187112
min,6.920000,7.760000,264.000000,447.000000,0.000000,0.000000,0.000000,0.000000,-139.000000,-139.000000,0.000000,7.000000,125000.0,1.000000
25%,22.730000,29.650000,421.000000,611.000000,8.000000,1.000000,1.000000,0.000000,-113.000000,-113.000000,1.000000,7.000000,125000.0,1221.000000
50%,24.650000,36.120000,448.000000,675.000000,121.000000,28.000000,2.000000,1.000000,-99.000000,-99.000000,3.000000,7.000000,125000.0,4477.000000
75%,27.600000,43.030000,506.000000,858.000000,634.000000,159.000000,4.000000,5.000000,-79.000000,-79.000000,5.000000,8.000000,125000.0,11820.000000
max,583.720000,622.150000,24001.000000,21930.000000,65535.000000,46896.000000,13.000000,128.000000,-32.000000,-32.000000,7.000000,12.000000,125000.0,37271.000000


In [ ]:
df.columns

Index(['date_time', 'device_id', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR',
       'WIFI', 'BLE', 'rssi', 'channel_rssi', 'snr', 'gateway',
       'channel_index', 'spreading_factor', 'bandwidth', 'f_cnt'],
      dtype='object')

In [ ]:
df.date_time = pd.to_datetime(df.date_time)
# sort by date ascendin, get 80% percentil
df.sort_values(by='date_time', ascending=True).reset_index(drop=True)['date_time'].quantile(0.8)

Timestamp('2023-07-17 12:26:11')

In [ ]:
# Data Preprocessing
start_id = max(df[df['obs_num'] == 0].index.tolist()) + 1
df = df[start_id:].copy()
df['sn_value'] = df['sn_value'].astype(float)
df_train = df[df['year'] < 2000]
df_test = df[df['year'] >= 2000]

spots_train = df_train['sn_value'].to_numpy().reshape(-1, 1)
spots_test = df_test['sn_value'].to_numpy().reshape(-1, 1)

scaler = StandardScaler()
spots_train = scaler.fit_transform(spots_train).flatten().tolist()
spots_test = scaler.transform(spots_test).flatten().tolist()

KeyError: 'obs_num'